In [48]:
import os
from pathlib import Path
from deepface import DeepFace
import numpy as np

In [49]:
persons = []

In [50]:
try:
    os.mkdir("dataset")
    os.mkdir("dataset/train")
    os.mkdir("dataset/test")
except FileExistsError:
    pass

In [51]:
TEST_SIZE = 0.333

In [54]:
for dir_ in os.listdir('lfw/'):
    folder = Path('lfw/' + dir_)
    num_photos = len(list(folder.iterdir()))
    
    # Если изображений больше, чем 2 отберем их для train/test выборки
    if num_photos > 2:
        z = 0
        
        # Сколько изображений взять для теста
        n_tests_images = int(num_photos * TEST_SIZE)
        n_tests_images = 1 if n_tests_images==0 else n_tests_images
        
        persons.append(dir_)
        
        for img in os.listdir('lfw/' + dir_):
            src = Path(f'lfw/{dir_}/{img}')
            
            # Распределение файлов в зависимсоти от размера test части
            if z < n_tests_images:
                dest_to_test = Path(f'dataset/test/{img}')
                dest_to_test.write_bytes(src.read_bytes())
                z += 1
            else:
                dest_to_train = Path(f'dataset/train/{img}')
                dest_to_train.write_bytes(src.read_bytes())

In [57]:
for filename in os.listdir('dataset/test'):
    path = f'dataset/test/{filename}'
    embedding = DeepFace.represent(img_path = path)
    print(np.array(embedding).shape)
    break

1/1 [==============================] - 0s 185ms/step
(2622,)
